# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [24]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [25]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [26]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [27]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [28]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [29]:
def simulate_single_flow(g, s, t, func_traj, difusao):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            try:
                # Escolhe aleatoriamente um dos sucessores.
                m = func_traj(g, n) ##
            except IndexError:
                continue
            
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

            
        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [30]:
def simulate_successful_flow(g, s, t, func_traj, difusao):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, func_traj, difusao)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [31]:
def simulate_all_flows(g,func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, func_traj, difusao)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não precisa entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [59]:
# TIMES = 1000
# func_traj = random_geodesic_successor
# difusao = False

def simul(TIMES, func_traj, difusao, coluna):
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, func_traj, difusao)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    df = pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
    'closeness analítico': [cc[n] for n in g.nodes],
    'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
    'betweenness analítico': [bc[n] for n in g.nodes],
    })
    return df.sort_values(coluna, ascending=0)

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [57]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

Construção de data frame só para comparar mais facilmente.

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [60]:
simul(100, random_geodesic_successor, False, "closeness simulado")

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.521429,0.560000,0.560000,medici
2,0.212454,0.211648,0.482759,0.482759,albizzi
7,0.091575,0.090220,0.482759,0.482759,tornabuon
9,0.086081,0.088022,0.482759,0.482759,ridolfi
3,0.260073,0.258352,0.466667,0.466667,guadagni
13,0.115385,0.115275,0.437500,0.437500,barbadori
11,0.075092,0.076923,0.424242,0.424242,strozzi
8,0.120879,0.121209,0.400000,0.400000,bischeri
5,0.142857,0.142857,0.388889,0.388889,salviati
14,0.087912,0.087912,0.388889,0.388889,castellan


In [61]:
simul(100, random_geodesic_successor, True,"closeness simulado")

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,1.389231,0.560000,0.560000,medici
2,0.212454,0.574396,0.482759,0.482759,albizzi
7,0.091575,0.243846,0.482759,0.482759,tornabuon
9,0.086081,0.249615,0.482759,0.482759,ridolfi
3,0.260073,0.691209,0.466667,0.466667,guadagni
13,0.115385,0.317143,0.437500,0.437500,barbadori
11,0.075092,0.205495,0.424242,0.424242,strozzi
8,0.120879,0.315604,0.400000,0.400000,bischeri
5,0.142857,0.406593,0.388889,0.388889,salviati
14,0.087912,0.230275,0.388889,0.388889,castellan


In [62]:
simul(100, random_path_successor, False,"closeness simulado")

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.620385,0.560000,0.304775,medici
3,0.260073,0.404341,0.466667,0.267810,guadagni
7,0.091575,0.328187,0.482759,0.261195,tornabuon
2,0.212454,0.228901,0.482759,0.261173,albizzi
9,0.086081,0.368791,0.482759,0.254076,ridolfi
14,0.087912,0.381044,0.388889,0.251463,castellan
8,0.120879,0.372802,0.400000,0.249869,bischeri
5,0.142857,0.142857,0.388889,0.242987,salviati
13,0.115385,0.235495,0.437500,0.241117,barbadori
11,0.075092,0.373626,0.424242,0.234843,strozzi


In [37]:
simul(100, random_path_successor, True)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.395549,0.333333,0.230875,ginori
1,0.000000,0.401154,0.325581,0.234153,lambertes
2,0.212454,0.619341,0.482759,0.256585,albizzi
3,0.260073,0.747088,0.466667,0.261235,guadagni
4,0.000000,0.259835,0.285714,0.203728,pazzi
5,0.142857,0.438462,0.388889,0.225035,salviati
6,0.521978,0.821319,0.560000,0.265739,medici
7,0.091575,0.682802,0.482759,0.280067,tornabuon
8,0.120879,0.650055,0.400000,0.263743,bischeri
9,0.086081,0.700495,0.482759,0.275368,ridolfi


In [38]:
simul(100, random_trail_successor, False)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.221354,ginori
1,0.000000,0.000000,0.325581,0.206872,lambertes
2,0.212454,0.264670,0.482759,0.272693,albizzi
3,0.260073,0.411923,0.466667,0.262922,guadagni
4,0.000000,0.000000,0.285714,0.198020,pazzi
5,0.142857,0.142857,0.388889,0.246858,salviati
6,0.521978,0.698242,0.560000,0.284275,medici
7,0.091575,0.321593,0.482759,0.251527,tornabuon
8,0.120879,0.345824,0.400000,0.227235,bischeri
9,0.086081,0.331264,0.482759,0.246710,ridolfi


In [39]:
simul(100, random_trail_successor, True)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.372473,0.333333,0.220371,ginori
1,0.000000,0.400275,0.325581,0.226326,lambertes
2,0.212454,0.692198,0.482759,0.249446,albizzi
3,0.260073,0.896923,0.466667,0.250622,guadagni
4,0.000000,0.265989,0.285714,0.201157,pazzi
5,0.142857,0.423516,0.388889,0.217664,salviati
6,0.521978,1.259286,0.560000,0.253784,medici
7,0.091575,0.932363,0.482759,0.263403,tornabuon
8,0.120879,0.784560,0.400000,0.248925,bischeri
9,0.086081,0.928516,0.482759,0.260342,ridolfi


In [40]:
simul(100, random_walk_successor, False)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.744066,0.333333,0.039480,ginori
1,0.000000,0.750989,0.325581,0.036411,lambertes
2,0.212454,2.436923,0.482759,0.035790,albizzi
3,0.260073,3.290275,0.466667,0.035479,guadagni
4,0.000000,0.717418,0.285714,0.040738,pazzi
5,0.142857,1.554341,0.388889,0.036682,salviati
6,0.521978,5.096154,0.560000,0.036687,medici
7,0.091575,2.481978,0.482759,0.035678,tornabuon
8,0.120879,2.445385,0.400000,0.035290,bischeri
9,0.086081,2.496429,0.482759,0.034608,ridolfi


In [41]:
simul(100, random_walk_successor, True)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.812967,0.333333,0.140852,ginori
1,0.000000,0.708132,0.325581,0.142263,lambertes
2,0.212454,3.103846,0.482759,0.159211,albizzi
3,0.260073,4.522747,0.466667,0.158894,guadagni
4,0.000000,0.758626,0.285714,0.126063,pazzi
5,0.142857,1.829505,0.388889,0.147909,salviati
6,0.521978,6.978681,0.560000,0.168979,medici
7,0.091575,2.445989,0.482759,0.170337,tornabuon
8,0.120879,2.796703,0.400000,0.160248,bischeri
9,0.086081,2.571319,0.482759,0.165551,ridolfi
